### Reading dataset:

In [201]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        path = os.path.join(dirname, filename)
        print(path)
        dataset = pd.read_csv(path)

/kaggle/input/dataset3/LREC_EmoInHindi.csv


### Installing dependencies:

In [202]:
!pip install torch
!pip install transformers

### Dataset overview

In [203]:
data = dataset
data.head()

,dialogueId,utterance_no,authorRole,utterance,emotions,emoIntensity
0,0,0,bot,"नमस्ते, सुरक्षित वातावरण में आपकी मदद करने के ...","confident,anticipation","1,1"
1,0,1,user,पिछले शुक्रवार को तीन नकाबपोशों ने मेरी पत्नी ...,"sad,anger","3,3"
2,0,2,bot,"इतना घिनौना, क्या मैं जान सकता हूँ कि मैं किसक...","sad,anticipation","3,1"
3,0,3,user,मैं हूं अरुण सिन्हा। मैं चाहता हूं कि उन राक्ष...,"anger,confident","3,2"
4,0,4,bot,"ठीक है, अरुण सिन्हा, कृपया मुझे घटना की तारीख,...",anticipation,1


### Function to calculate metric

In [204]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
def calculate_metrics(y_true, y_pred, y_intensity_true, y_intensity_pred):
    emotion_accuracy = accuracy_score(y_true, y_pred)
    emotion_recall = recall_score(y_true, y_pred, average='macro')  
    emotion_precision = precision_score(y_true, y_pred, average='macro')
    emotion_f1 = f1_score(y_true, y_pred, average='macro')

    intensity_accuracy = accuracy_score(y_intensity_true, y_intensity_pred)
    intensity_recall = recall_score(y_intensity_true, y_intensity_pred, average='macro') 
    intensity_precision = precision_score(y_intensity_true, y_intensity_pred, average='macro')
    intensity_f1 = f1_score(y_intensity_true, y_intensity_pred, average='macro')

    return {
        'emotion_accuracy': emotion_accuracy,
        'emotion_recall': emotion_recall,
        'emotion_precision': emotion_precision,
        'emotion_f1': emotion_f1,
        'intensity_accuracy': intensity_accuracy,
        'intensity_recall': intensity_recall,
        'intensity_precision': intensity_precision,
        'intensity_f1': intensity_f1
    }
# y_pred = [1, 0, 0] 
# y_true = [1, 0, 0]
# y_intensity_true = [4, 2, 0] 
# y_intensity_pred = [4, 2, 0]
# result = calculate_metrics(y_true, y_pred, y_intensity_true, y_intensity_pred)
# print(result)

### Cloning the fasttext tokenizer from github, and defination to convert sentences into embedding:

In [205]:
!git clone https://github.com/facebookresearch/fastText.git
!python /kaggle/working/fastText/download_model.py hi

import fasttext
ft = fasttext.load_model('/kaggle/working/cc.hi.300.bin')

fatal: destination path 'fastText' already exists and is not an empty directory.
File exists. Use --overwrite to download anyway.


In [206]:
def text_to_fasttext_embeddings(text):
    embeddings =  ft.get_word_vector(text)
    return embeddings

# temp = text_to_fasttext_embeddings("आज के समय में तकनीकी उन्नति ने हमारे जीवन को गहराई से प्रभावित किया है।")
# print(temp)

### Data Preprocessing:

In [207]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

mapping = {
    'joy': 0,
    'anticipation': 1,
    'neutral': 2,
    'anger': 3,
    'disgusted': 4,
    'confident': 5,
    'annoyed': 6,
    'hopeful': 7,
    'apprehensive': 8,
    'grateful': 9,
    'sad': 10,
    'compassion': 11,
    'fear': 12,
    'guilty': 13,
    'surprised': 14,
    'impressed': 15,
    'confused': 16
}

def convert_to_float(embedding):
    return [float(value) for value in embedding]

def preprocess_data(data,batch_size):
    # Initialize lists for corpus, emotions, and intensities
    corpus = []
    emotions = []
    intensities = []

    # Iterate through the DataFrame
    for index, row in data.iterrows():
        # Extract utterance
        utterance = row['utterance']
        corpus.append(utterance)

        # Extract emotions and intensities
        emo = row['emotions'].split(',')
        emo_intensities = row['emoIntensity'].split(',')

        utterance_emotions = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        utterance_intensities = [[0, 0, 0, 0],[0, 0, 0, 0],[0, 0, 0, 0],[0, 0, 0, 0],[0, 0, 0, 0],
                                [0, 0, 0, 0],[0, 0, 0, 0],[0, 0, 0, 0],[0, 0, 0, 0],[0, 0, 0, 0],
                                [0, 0, 0, 0],[0, 0, 0, 0],[0, 0, 0, 0],[0, 0, 0, 0],[0, 0, 0, 0],
                                [0, 0, 0, 0],[0, 0, 0, 0]]

        for idx, intensity in zip(emo,emo_intensities):
            if idx in mapping and intensity !='' and int(intensity)<4:
                utterance_emotions[mapping[idx]] = 1
#                 print(int(intensity))
                utterance_intensities[mapping[idx]][int(intensity)] = 1

        emotions.append(utterance_emotions)
        intensities.append(utterance_intensities)
    
    text = [text_to_fasttext_embeddings(sentence) for sentence in corpus]
    # Convert text embeddings to float
    text_float = [convert_to_float(embedding) for embedding in text]

    # Convert data to PyTorch tensors
    text_tensor = torch.tensor(text_float, dtype=torch.float32)
    emotions_tensor = torch.tensor(emotions, dtype=torch.float32)
    intensities_tensor = torch.tensor(intensities, dtype=torch.float32)

    # Creating a TensorDataset
    dataset = TensorDataset(text_tensor, emotions_tensor, intensities_tensor)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    
    return dataloader

### Splitting train, test and validation data:

In [208]:
from sklearn.model_selection import train_test_split

train_val_df, test_df = train_test_split(data, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_val_df, test_size=0.2, random_state=42)

train_dataloader = preprocess_data(train_df,batch_size=32)
test_dataloader = preprocess_data(test_df,batch_size=32)
val_dataloader = preprocess_data(val_df,batch_size=32)

### Fnet Block Implementation:

In [209]:
import torch
from torch import nn

class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout = 0.03):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)
      
class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn
    def forward(self, x, **kwargs):
        return self.fn(self.norm(x), **kwargs)

class FNetBlock(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        x = x.unsqueeze(0) if x.dim() == 1 else x      #extra added
        x = torch.fft.fft(torch.fft.fft(x, dim=-1), dim=-2).real
        return x.squeeze(0) if x.dim() == 3 else x     #extra added
#         return x

class FNet(nn.Module):
    def __init__(self, dim, depth, mlp_dim, dropout = 0.):
        super().__init__()
        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                PreNorm(dim, FNetBlock()),
                PreNorm(dim, FeedForward(dim, mlp_dim, dropout = dropout))
            ]))
    def forward(self, x):
        for attn, ff in self.layers:
            x = attn(x) + x
            x = ff(x) + x
        return x

### Defining the Network:

In [210]:
import torch
from torch import nn
from torch.nn import functional as F

class EmotionIntensityModel(nn.Module):
    def __init__(self, embedding_dim, num_emotions, num_intensities, transformer):
        super().__init__()
        self.transformer = transformer
        self.fc1 = nn.Linear(embedding_dim, num_emotions)
        self.fc2 = nn.Linear(embedding_dim, num_emotions * num_intensities)

    def forward(self, x):
        x = self.transformer(x)
        emotions = F.softmax(self.fc1(x), dim=1)
        intensities = self.fc2(x).view(x.shape[0], -1, 4)  # Reshape to (batch_size, num_emotions, num_intensities)
        return emotions, intensities

# Initialize the FNet transformer
fnet_transformer = FNet(dim=300, depth=2, mlp_dim=300, dropout=0.03)

In [211]:
# #  # Model
# class FNetEmotionClassifier(nn.Module):
#     def __init__(self, embedding_dim=300, num_emotions=17):
#         super().__init__()

#         self.fnet = FNet(dim=embedding_dim, depth=2, mlp_dim=embedding_dim * 2, dropout=0.3)  

#         self.emotion_classifier = nn.Sequential(
#             nn.Linear(embedding_dim, num_emotions),
#             nn.Softmax(dim=-1)  # Softmax for multi-label emotion output
#         )

# #         self.intensity_classifier = nn.Sequential(
# #             nn.Linear(embedding_dim, intensity_classes),
# #             nn.LogSoftmax(dim=-1)  # LogSoftmax for intensity classification
# #         )

#     def forward(self, input_ids):
#         fnet_output = self.fnet(input_ids)
# #         print("fnet_output in cls FNetEmotionClassifier: ",fnet_output)
#         emotion_logits = self.emotion_classifier(fnet_output)
# #         intensity_logits = self.intensity_classifier(fnet_output)
#         return emotion_logits

### Loss function for NLL:

In [212]:
import torch

class NLLLossCustom:
    def __init__(self):
        pass
    
    def __call__(self, intensity_logits, intensities):
        # Compute log probabilities
        log_probs = torch.log_softmax(intensity_logits, dim=1)
        
        # Reshape intensities to [batch_size * num_classes]
        num_samples, num_classes = log_probs.size()
        intensities = intensities.view(num_samples, num_classes)
        
        # Compute the negative log likelihood loss
        loss = -torch.sum(intensities * log_probs) / num_samples
        
        return loss

### Processing Predicted Tensors for calculating accuracy:

In [213]:
def convert_to_int(list_of_floats):
    return [int(num) for num in list_of_floats]

def aligne_predictions(emotions,emotion_logits,intensities,intensity_logits):
    """ used to align the predictions and the real values for the metric calculations"""
    emotions_actual = emotions.tolist()
    emotions_predicted = np.array(emotion_logits.tolist())
    intensity_predicted = intensity_logits.tolist()
    intensity_actual = intensities.tolist()
#     print("emotions_actual: ",emotions_actual)
#     print("emotions_predicted: ",emotions_predicted)
#     print("intensity_actual: ",intensity_actual)
#     print("intensity_predicted: ",intensity_predicted)
    
    emo_pred = []
    emo_true = []
    intensity_pred = []
    intensity_true = []
    for current in range(len(emotions_actual)):
        emo_pred.extend(emotions_predicted[current])
        emo_true.extend(emotions_actual[current])
        temp_pred = []
        temp_actual = []
        for each in range(len(intensity_actual[current])):
            temp_pred.append(max(range(len(intensity_predicted[current][each])), key=intensity_predicted[current][each].__getitem__))
            temp_actual.append(max(range(len(intensity_predicted[current][each])), key=intensity_actual[current][each].__getitem__))
        intensity_pred.extend(temp_pred)
        intensity_true.extend(temp_actual)

        emo_pred = [int(elem) for elem in emo_pred]
        emo_true = [int(elem) for elem in emo_true]
        intensity_pred = [int(elem) for elem in intensity_pred]
        intensity_true = [int(elem) for elem in intensity_true]

        return emo_pred,emo_true,intensity_pred,intensity_true

### Intializing Model:

In [214]:
!export CUDA_LAUNCH_BLOCKING=1

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize the EmotionIntensityModel with the FNet transformer
model = EmotionIntensityModel(embedding_dim=300, num_emotions=17, num_intensities=4, transformer=fnet_transformer)
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.003)
loss_fn_emotions = nn.MultiLabelSoftMarginLoss()
loss_fn_intensity = nn.CrossEntropyLoss()
# loss_fn_intensity = NLLLossCustom()

### defining Trainer:

In [215]:
import time
def train(model, dataloader, optimizer, loss_fn_emotions, loss_fn_intensity, device):
    model.train()
    total_loss = 0.0
    all_emotion_preds, all_emotion_true = [], []
    all_intensity_preds, all_intensity_true = [], []

    for batch in dataloader:
        start_time = time.time()  # Capture start time for the epoch 

        input_ids = batch[0].to(device)
        emotions = batch[1].to(device)  # Assuming labels are properly encoded
        intensities = batch[2].to(device)

        optimizer.zero_grad()
        emotion_logits, intensity_logits = model(input_ids)
#         print("intensity_logits: ",intensity_logits.shape)
# #         print("intensities: ",intensities.shape)

#         num_samples = emotion_logits.size(0)  # Get the number of samples
#         num_classes = intensity_logits.size(1)  # Get the number of classes
#         intensities = intensities.view(num_samples, num_classes)
            
        loss_emotion = loss_fn_emotions(emotion_logits, emotions)
        loss_intensity = loss_fn_intensity(intensity_logits.view(-1, 4), intensities.view(-1, 4))
        total_loss = loss_emotion + loss_intensity

#         emotion_pred,emotion_true,intensity_pred,intensity_true = aligne_predictions(
#                 emotions,emotion_logits,intensities,intensity_logits)
            
#         all_emotion_preds.extend(emotion_pred)
#         all_emotion_true.extend(emotion_true)
#         all_intensity_preds.extend(intensity_pred)
#         all_intensity_true.extend(intensity_true)
            
        total_loss.backward()
        optimizer.step()

#     epoch_metrics = calculate_metrics(all_emotion_true, all_emotion_preds,
#                                       all_intensity_true, all_intensity_preds)
    epoch_time = time.time() - start_time

#     print(f"Epoch Training Loss: {total_loss:.3f}, Time: {epoch_time:.2f}s")
    print("="*60)
    print(f"Epoch Training Loss: {total_loss:.3f}")
    print(f"Epoch Time: {epoch_time:.2f}")
    print("="*60)

### defining evaluator:

In [216]:
def evaluate(model, dataloader, loss_fn_emotions, loss_fn_intensity, device):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0.0
    all_emotion_preds, all_emotion_true = [], []
    all_intensity_preds, all_intensity_true = [], []

    with torch.no_grad():  # Disable gradient calculation for efficiency
        for batch in dataloader:
            input_ids = batch[0].to(device)   # input_ids
            emotions = batch[1].to(device)    # emotions
            intensities = batch[2].to(device)    # intensities

            emotion_logits, intensity_logits = model(input_ids)
            
            loss_emotion = loss_fn_emotions(emotion_logits, emotions)
            loss_intensity = loss_fn_intensity(intensity_logits.view(-1, 4), intensities.view(-1, 4))
            total_loss = loss_emotion + loss_intensity
        
            emotion_pred,emotion_true,intensity_pred,intensity_true = aligne_predictions(
                emotions,emotion_logits,intensities,intensity_logits)
#             print("emotion_pred: ",emotion_pred)
#             print("emotion_true: ",emotion_true)
#             print("intensity_pred: ",intensity_pred)
#             print("intensity_true: ",intensity_true)

            all_emotion_preds.extend(emotion_pred)
            all_emotion_true.extend(emotion_true)
            all_intensity_preds.extend(intensity_pred)
            all_intensity_true.extend(intensity_true)
            
    metrics = calculate_metrics(all_emotion_true, all_emotion_preds,
                                all_intensity_true, all_intensity_preds)
    return total_loss / len(dataloader), metrics  # Average loss over the validation set

### Training:

In [217]:
for epoch in range(20):
    train(model, train_dataloader, optimizer, loss_fn_emotions, loss_fn_intensity, device)   #change dataloader
    val_loss, val_metrics = evaluate(model, val_dataloader, loss_fn_emotions, loss_fn_intensity, device)
    print(f"Epoch {epoch+1} Validation Loss: {val_loss:.3f}, Metrics: {val_metrics} \n") 

Epoch Training Loss: 0.981
Epoch Time: 0.01
Epoch 1 Validation Loss: 0.006, Metrics: {'emotion_accuracy': 0.9017833377694969, 'emotion_recall': 0.6036222577893637, 'emotion_precision': 0.6407489130151788, 'emotion_f1': 0.618132687675342, 'intensity_accuracy': 0.11019430396593026, 'intensity_recall': 0.41035779560243996, 'intensity_precision': 0.2767602368166599, 'intensity_f1': 0.09015252158807135} 

Epoch Training Loss: 1.387
Epoch Time: 0.01
Epoch 2 Validation Loss: 0.007, Metrics: {'emotion_accuracy': 0.9182858663827522, 'emotion_recall': 0.6331595335311224, 'emotion_precision': 0.7047474729225571, 'emotion_f1': 0.6590475019310509, 'intensity_accuracy': 0.09502262443438914, 'intensity_recall': 0.351381256016276, 'intensity_precision': 0.26609848678666326, 'intensity_f1': 0.07477769896689059} 

Epoch Training Loss: 1.045
Epoch Time: 0.01
Epoch 3 Validation Loss: 0.004, Metrics: {'emotion_accuracy': 0.8999201490550972, 'emotion_recall': 0.6014413836112223, 'emotion_precision': 0.64761

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 14 Validation Loss: 0.004, Metrics: {'emotion_accuracy': 0.9241416023422944, 'emotion_recall': 0.6151911617495212, 'emotion_precision': 0.7340976168450729, 'emotion_f1': 0.649254956067811, 'intensity_accuracy': 0.035666755389938784, 'intensity_recall': 0.35206274817619354, 'intensity_precision': 0.02615855027430094, 'intensity_f1': 0.04831124352038056} 

Epoch Training Loss: 0.807
Epoch Time: 0.01
Epoch 15 Validation Loss: 0.004, Metrics: {'emotion_accuracy': 0.8876763375033271, 'emotion_recall': 0.5519104287652232, 'emotion_precision': 0.5731550471894424, 'emotion_f1': 0.5593513062812674, 'intensity_accuracy': 0.0896992281075326, 'intensity_recall': 0.3587296163909915, 'intensity_precision': 0.27518853695324286, 'intensity_f1': 0.07610943212299216} 

Epoch Training Loss: 0.755
Epoch Time: 0.01
Epoch 16 Validation Loss: 0.004, Metrics: {'emotion_accuracy': 0.8983231301570402, 'emotion_recall': 0.6003546474305455, 'emotion_precision': 0.6384332579185521, 'emotion_f1': 0.6149582005

### Testing:

In [218]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  
average_loss, metrics = evaluate(model, test_dataloader, loss_fn_emotions, loss_fn_intensity, device)
print("average_loss: ",average_loss)
print("metrics: ", metrics)

average_loss:  tensor(0.0030, device='cuda:0')
metrics:  {'emotion_accuracy': 0.9230605285592498, 'emotion_recall': 0.5947040482622737, 'emotion_precision': 0.7233088614983947, 'emotion_f1': 0.6264284247622959, 'intensity_accuracy': 0.0907928388746803, 'intensity_recall': 0.4105724759477489, 'intensity_precision': 0.2743574443100086, 'intensity_f1': 0.07809623687745736}


### To view output in a sentence:

In [219]:
rev_mapping = {
    0: 'joy', 
    1: 'anticipation', 
    2: 'neutral', 
    3: 'anger', 
    4: 'disgusted', 
    5: 'confident', 
    6: 'annoyed', 
    7: 'hopeful', 
    8: 'apprehensive', 
    9: 'grateful', 
    10: 'sad', 
    11: 'compassion', 
    12: 'fear', 
    13: 'guilty', 
    14: 'surprised', 
    15: 'impressed', 
    16: 'confused',
    17: 'emotions'
}

### Custom testing for a sentence:

In [225]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  
model.to(device)
model.eval()
sentence = "आपके साथ समय बिताने के लिए मुझे धन्यवाद।"
input_ids = text_to_fasttext_embeddings(sentence)
input_ids = convert_to_float(input_ids)
input_ids = torch.tensor(input_ids, dtype=torch.float32).to(device)
emotion_logits, intensity_logits = model(input_ids)
# aligne_predictions(intensity_logits,intensity_logits)
emotions_predicted = np.array(emotion_logits.tolist())[0]
intensity_predicted = intensity_logits.tolist()[0]

# print(emotions_predicted)
# print(intensity_predicted)
k = len(emotions_predicted)
emotion_pred = np.argsort(emotions_predicted)[-4:][::-1]
# print(emotion_pred)
intensity_pred = [max(range(len(intensity_predicted[index])), key=intensity_predicted[index].__getitem__) for index in emotion_pred]

print("Emotion",": ","Intensity")
print("="*15)
emotion_pred = [rev_mapping[i] for i in emotion_pred]
for i,j in zip(emotion_pred,intensity_pred):
    print(i,": ",j)

Emotion :  Intensity
anticipation :  2
neutral :  0
confident :  2
anger :  2
